###  Mount Google Drive

In [ ]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3


Mounted at /content/drive


### Clone Codebase

In [ ]:
! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3
! cd /content/drive/MyDrive/COMP4901B-Homework3 && git clone https://github.com/hkust-nlp/COMP4901B-LLMs.git

### Download Dataset

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/setup.sh

### Q3 Start The Tuning

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3/ && bash scripts/self_train_gsm8k.sh

### Part 3

#### Evaluate base model

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ifeval && bash run.sh ../SmolLM2-135M ./results/SmolLM2-135M

#### My Hyperparameter Tuning 1 in Q6

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ifeval && bash run.sh ../ckpt/FT1 ./results/SmolLM2-135M-SFT-1

#### My Hyperparameter Tuning 2

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ && bash scripts/sft.sh

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ifeval && bash run.sh ../ckpt/FT2 ./results/SmolLM2-135M-SFT-2

#### My Hyperparameter Tuning 3




In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ && bash scripts/sft.sh

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ifeval && bash run.sh ../ckpt/FT3 ./results/SmolLM2-135M-SFT-3

#### My Hyperparameter Tuning 4




In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ && bash scripts/sft.sh

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ifeval && bash run.sh ../ckpt/FT4 ./results/SmolLM2-135M-SFT-4

#### My Hyperparameter Tuning 5


In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ && bash scripts/sft.sh

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework2/COMP4901B-LLMs/assignment2/ifeval && bash run.sh ../ckpt/FT4 ./results/SmolLM2-135M-SFT-5